In [1]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Flatten

df = pd.read_csv('imdb_master.csv',encoding='latin-1')
df = df.drop(['Unnamed: 0'],axis=1)
df.head(5)

,type,review,label,file
0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [2]:
df = df[df['label']!='unsup']
sentences = df['review'].values
y = df['label'].values
#tokenizing data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(sentences)
#getting the vocabulary of data
# sentences = tokenizer.texts_to_matrix(sentences)
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [3]:
#tokenizing data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(sentences)
#getting the vocabulary of data
max_review_len = max([len(s.split()) for s in sentences])
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

123682


In [4]:
X_train, X_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)
padded_train = pad_sequences(X_train_tokens,maxlen=max_review_len)
paded_test = pad_sequences(X_test_tokens,maxlen=max_review_len)

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_review_len))
# model.add(Flatten())
model.add(layers.Dense(300, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
history=model.fit(padded_train,y_train, epochs=5, verbose=True, validation_data=(paded_test,y_test), batch_size=256)

Epoch 1/5
147/147 [==============================] - 345s 2s/step - loss: 0.6928 - acc: 0.5048 - val_loss: 0.6925 - val_acc: 0.5052
Epoch 2/5
147/147 [==============================] - 371s 3s/step - loss: 0.6924 - acc: 0.5090 - val_loss: 0.6924 - val_acc: 0.5023
Epoch 3/5
147/147 [==============================] - 365s 2s/step - loss: 0.6923 - acc: 0.5059 - val_loss: 0.6923 - val_acc: 0.5053
Epoch 4/5
147/147 [==============================] - 365s 2s/step - loss: 0.6923 - acc: 0.5020 - val_loss: 0.6923 - val_acc: 0.5053
Epoch 5/5
147/147 [==============================] - 364s 2s/step - loss: 0.6923 - acc: 0.5050 - val_loss: 0.6923 - val_acc: 0.5053


In [11]:
test_loss, test_acc = model.evaluate(paded_test, y_test)
print(test_acc)

391/391 [==============================] - 14s 36ms/step - loss: 0.3802 - acc: 0.8649
0.8648800253868103
